In [2]:
# !pip install python-dotenv
from dotenv import load_dotenv
import os
# os.getcwd()

load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")
alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

In [12]:
# Homework
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, WikipediaQueryRun
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper, TextRequestsWrapper

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

class NomadWikiSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for in wiki. Example query: Research about the XZ backdoor."
    )

class NomadWikiSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = """
    Use this tool to find the information of the term at Wikipedia.
    It takes a query as an argument.
    """
    args_schema: Type[
        NomadWikiSearchToolArgsSchema
    ] = NomadWikiSearchToolArgsSchema

    def _run(self, query):
        wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wikipedia.run(query)

class DuckDuckGoSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for in DuckDuck. Example query: Research about the XZ backdoor."
    )

class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckSearchTool"
    description = """
    Use this tool to find the information of the term.
    It takes a query as an argument.
    It gives output URL from the search results.
    """
    args_schema: Type[
        DuckDuckGoSearchToolArgsSchema
    ] = DuckDuckGoSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)

class RequestSaveToolArgsSchema(BaseModel):
    query: str = Field(
        description="If the DuckDuckSearchTool returns URL request for the contents and save it as .txt. Example query: "
    )

class RequestSaveTool(BaseTool):
    name = "RequestSaveTool"
    description = """
    Use this tool to scrape the information of the web URL which DuckDuckGoSearchTool returns,
    It takes a url as an argument and save it as .txt file.
    """
    args_schema: Type[
        RequestSaveToolArgsSchema
    ] = RequestSaveToolArgsSchema

    def _run(self, query):
        requests = TextRequestsWrapper()
        f = open('homework12.txt','w', encoding='utf-8')
        f.write(requests.get(query))
        f.close()
        return requests.get(query)

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        NomadWikiSearchTool(),
        # DuckDuckGoSearchTool(), # 덕덕고 HTML 오류 생김
        RequestSaveTool(),
    ],
)

prompt = """Research about the XZ backdoor at Wiki.
And return the URL from the search result.
Input the URL to request and scrape the information from the URL"""
# Input the URL to scrape the information and save it as .txt file."""

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: On 29 March 2024, software developer Andres Freund reported that he had found a maliciously introduced backdoor in the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 released in February 2024.While xz is commonly present in most Linux distributions, the backdoor only targeted Debian- and RPM-based systems running on the x86-64 architecture. At the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions.The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been assigned a CVSS score of 10.0, the highest possible score.

Page: XZ Utils
Summary: XZ Utils (previously LZMA Utils) is a set of free software command-line lossless da

InvalidRequestError: This model's maximum context length is 16385 tokens. However, your messages resulted in 39743 tokens (39569 in the messages, 174 in the functions). Please reduce the length of the messages or functions.